<a href="https://colab.research.google.com/github/YooNayoung/ESAA/blob/main/0509session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[NLP 언제까지 미룰래? 일단 들어와!!] #4. word embedding**

## **Embedding이란?**
- vectorization의 치명적인 단점은 바로 단어나 문장들 사이의 관계에 대해서 설명하지 못한다는 것.
- 이 때 사용되는 것이 Embedding 기법이며 word2vec을 비롯한 다양한 임베딩 기법들이 존재함.
- 비슷한 의미를 내포하고 있는 토큰들은 서로 가깝게, 그렇지 않은 토큰들은 서로 멀리 뿌리도록 하는 것이 임베딩의 목적. 검색 시스템, 감성 분석 등에서는 훌륭한 임베딩을 수행하는 것이 전체 문제 해결에 많은 영향을 줌.
- 임베딩 또한 하나의 모델을 의미하며 훈련이 필요. 데이터가 충분하고 시간이 많으면 소지한 데이터에 특화된 임베딩 모델을 학습시킬 수 있음. 보통은 pre_trained embedding model을 가져와서 사용함.

## **1.Keras Embedding Layer**
- 기본적으로 가장 쉽고 빠르게 네트워크 모델에 임베딩 층을 주입할 수 있는 방식. 
- 무작위로 특정 차원으로 입력 벡터들을 뿌린 후 학습을 통해 가중치들을 조정해 나감. 즉, 단어 사이의 관계를 반영하는 방법이 아님.

In [21]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/ESAA(OB)/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ESAA(OB)/test_x.csv')
submission = pd.read_csv('/content/drive/MyDrive/ESAA(OB)/sample_submission.csv', index_col=False)

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=100):
    
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer

train_X, vocab_size, tokenizer = text2sequence(train['text'], max_len = 100)

vocab_size :  42331


In [30]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding

In [31]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length = 100))

## **2. word2vec**
- word2vec의 핵심 아이디어는 "친구를 보면 그 사람을 알 수 있다"입니다. 주변 단어와의 관계를 통해 해당 단어의 의미적 특성을 파악함.
- word2vec embedding matrix를 kears의 embedding에 주입하는 과정은 다음과 같음.

1. 구글의 사전 훈련된 word2vec bin 파일을 다운로드

2. gensim 모듈과 bin파일을 활용해 word2vec 모델을 로드

In [19]:
import gensim  
word2vec = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/ESAA(OB)/GoogleNews-vectors-negative300.bin.gz', binary = True)

3. vocabulary에 있는 토큰들의 벡터를 가져와 embedding matrix에 저장

In [36]:
embedding_matrix = np.zeros((vocab_size, 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(tokenizer.word_index): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in word2vec: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = word2vec[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[index] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.
    else:
        print("word2vec에 없는 단어입니다.")
        break

word2vec에 없는 단어입니다.


4. keras embedding layer에 embedding_matrix를 가중치로 주어 이용합니다.

In [38]:
model = Sequential()
model.add(Embedding(vocab_size, 300,weights = [embedding_matrix], input_length = 100))

## **3. glove**
- glove는 word2vec의 단점을 보완하기 위하여 등장함. word2vec이 사용자가 지정한 주변 단어의 개수에 대해서만 학습이 이루어지기 때문에 데이터 전체에 대한 정보를 담기 어렵다는 단점을 지적하였음.
- glove의 핵심 아이디어는 "각 토큰들 간의 유사성은 그대로 가져가면서 데이터 전체에 대한 빈도를 반영하자"
- glove embedding matrix를 keras의 embedding matrix에 주입하는 방법은 다음과 같음.

1. 사전 훈련된 벡터를 갖고 있는 txt 파일 다운로드

2. txt 파일에 있는 단어와 벡터들을 glove dictionary에 저장

In [26]:
2. # load the whole embedding into memory
glove = dict()
f = open('/content/drive/MyDrive/ESAA(OB)/glove.txt')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.array(values[1:], dtype='float32')
    glove[word] = vector
f.close()

3. vocabulary에 있는 토큰들의 벡터를 가져와 embedding matrix에 저장

In [40]:
embedding_matrix = np.zeros((vocab_size, 100)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(tokenizer.word_index): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in glove: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = glove[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[index] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.
    else:
        print("glove 없는 단어입니다.")
        break

glove 없는 단어입니다.


4. keras embedding layer에 embedding_matrix를 가중치로 주어 이용

In [42]:
model = Sequential()
model.add(Embedding(vocab_size, 100,weights = [embedding_matrix], input_length = 100))

## **4. Fasttext**
- Fasttext의 핵심 아이디어는 단어 단위가 아닌 sub 단어를 단위로 사용한다는 것임.
- 따라서 미리 학습되지 않은 단어들에 대한 vector도 표현해준다는 장점이 있음.


1. 사전 훈련된 bin 파일 다운로드

2. vec 파일을 gensim을 활용하여 읽어옴.

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
FastText = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/ESAA(OB)/fasttext.bin', binary = True, encoding='latin_1')

3. vocabulary에 있는 토큰들의 벡터를 가져와 embedding matrix에 저장

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(tokenizer.word_index): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in word2vec: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = word2vec[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[i] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.

4. keras embedding layer에 embedding_matrix를 가중치로 주어 이용

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 300,weights = [embedding_matrix], input_length = 100))

# **[NLP 언제까지 미룰래? 일단 들어와!!] #5. Modeling(완)**

## **Modeling**
- 현재 ML에서 주로 사용되는 SVM, RF, Boosting Machine등을 활용하여 간단한 NLP 태스크를 해결이 가능하지만 복잡한 문제를 해결하기에는 한계가 있습니다. 따라서 기본적으로 네트워크 모델을 이용하여 NLP 문제를 해결하는 것이 일반적입니다.
- NLP 문제를 해결하기 위해서 주로 쓰이는 RNN과 Layer들의 간단한 구조를 소개해보겠습니다.

## **1.RNN**
- RNN은 순차적인 구조이며 그 특징 때문에 연산에 많은 시간이 걸리rp 되고 네트워크의 고질적인 문제인 역전파 소실 문제가 발생합니다.

## **2. LSTM**
- 이제 LSTM이 RNN 구조에서 특정 시점의 정보를 다음 시점으로 전달할 때 얼만큼의 정보를 전달할지 결정하는 역할을 하겠다는 것을 유추할 수 있습니다.
- LSTM의 가장 큰 특징은 기존 RNN에 cell state를 추가한 것입니다. 이 cell state는 입력들의 정보를 선별하여 다음 출력으로 내보내는 게이트 역할을 합니다. 이 과정을 통해 불필요한 정보들을 걸러내어 매끄러운 진행이 가능하고 이로 인해 역전파 소실 문제를 줄여 성능이 증가하게 됩니다. 

## **3. GRU**
- GRU는 LSTM의 복잡한 구조를 보다 간결하게 보완한 모델입니다. LSTM의 장점을 가져오면서 속도적인 부분을 개선하여 더욱 빠른 속도로 비슷한 성능을 낸다고 알려져 있습니다.

## **4. Attention**
- attention 매커니즘은 RNN 모델의 구조적인 한계를 극복한 모델입니다. 입력으로 이용된 정보들을 순차적으로 고정 길이로 압축하면서 발생하는 정보의 손실, 역전파 소실 문제를 해결하기 위해 노력했고 어떤 토큰의 정보가 가장 큰 도움을 줬는지 알 수 있습니다.
- 뒤이어 나온 자연어 처리의 세기적 발견인 Transformer(attention is all you need)의 핵심 알고리즘이며 이 둘에 대한 이해가 바탕이 되어야 다양한 문제에서 SOTA를 자랑하는 ELMO, BERT, GPT 등의 방법론들을 이해 할 수 있습니다.

## **5. 대회 적용**

### 간단한 전처리 + 형태소 분석

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [56]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
!pip install -U tensorflow-addons
!pip install -q "tqdm>=4.36.1"

In [ ]:
from konlpy.tag import Okt
import tqdm
import re
def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    tokenizer = Okt()
    
    for text in tqdm.tqdm(text_list):
        txt = re.sub('[^가-힣a-z]', ' ', text.lower())
        token = tokenizer.morphs(txt)
        token = [t for t in token if t not in stopwords or type(t) != float]
        
    return token, tokenizer

train['token'], okt = text_preprocessing(train['text'])

### vectorization

In [ ]:
def text2sequence(train_text, max_len=1000):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_text)
    train_X_seq = tokenizer.texts_to_sequences(train_text)
    vocab_size = len(tokenizer.word_index) + 1
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len)
    return X_train, vocab_size, tokenizer

train_y = train['info']
train_X, vocab_size, vectorizer = text2sequence(train['token'], max_len = 100)
print(train_X.shape, train_y.shape)

### Embedding

In [ ]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary = True)
embedding_matrix = np.zeros((vocab_size, 300))

for index, word in enumerate(vocabulary):
    if word in word2vec
        embedding_vector = word2vec[word] 
        embedding_mxtrix[i] = embedding_vector 
    else:
        print("word2vec에 없는 단어입니다.")
        break

### Modeling



In [ ]:
def LSTM(vocab_size, max_len=1000):
    model = Sequential()
    model.add(Embedding(vocab_size, 300,weights = [embedding_matrx], input_length = max_len)) #임베딩 가중치 적용 코드
    model.add(SpatialDropout1D(0.3))
    model.add(LSTM(64))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
    model.summary()
    return model